In [1]:
import pandas as pd


from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
from tqdm import tqdm

In [3]:
splits = pd.read_csv('../data/splits/data_splits.csv',
                     dtype={
    'book': 'string',
    'chapter': 'string',
    'split': 'string'
})

devs = splits[splits['split'] == 'DEV']
trains = splits[splits['split'] == 'TRAIN']
tests = splits[splits['split'] == 'TEST']

Pattern: /dev/{book}/{chapter}.frags.pnx.words.txt

In [4]:
base_path = '../data/splits'
dev_pattern = '/dev/{}/{}.frags.pnx.words.txt'
train_pattern = '/train/{}/{}.frags.pnx.words.txt'
test_pattern = '/test/{}/{}.frags.pnx.words.txt'

In [5]:
def get_all(pattern, splits):
  all = []
  for _, row in tqdm(splits.iterrows()):
    with open(base_path + pattern.format(row['book'], row['chapter'])) as f:
      all.extend([[' '.join(['#'.join(word.split('\t'))                       for word in frag.split('\n')]), row['book'], row['chapter']]                          for frag in f.read().split('\n\n')])
  return all

In [6]:
all_dev_aligned = get_all(dev_pattern, devs)
all_test_aligned = get_all(test_pattern, tests)
all_train_aligned = get_all(train_pattern, trains)

20it [00:13,  1.50it/s]
22it [00:12,  1.72it/s]
104it [00:51,  2.03it/s]


In [7]:
all_dev_aligned = pd.DataFrame(all_dev_aligned)
all_test_aligned = pd.DataFrame(all_test_aligned)
all_train_aligned = pd.DataFrame(all_train_aligned)

In [8]:
print(len(all_dev_aligned), len(all_test_aligned), len(all_train_aligned))

2989 3296 14436


In [9]:
import re

In [10]:
def prune_punctuation(fragment):
  all = fragment.split(' ')
  all = [a for a in all if not re.fullmatch(r'^[–•«»!,.:\u060C\u061B\u061F\u2026]+$', a.split('#')[0])]
  if len(all) == 0:
    return 'DELETEME'
  else:
    return ' '.join(all)

def cleanse(df):
  df[0] = df.apply(lambda row: prune_punctuation(row[0]), axis=1)
  df = df[df[0] != 'DELETEME']
  return df

In [11]:
all_dev_aligned = cleanse(all_dev_aligned)
all_train_aligned = cleanse(all_train_aligned)
all_test_aligned = cleanse(all_test_aligned)

In [12]:
base_path = '../data/'



all_dev_aligned.to_csv(base_path + 'all_dev_aligned.csv')

all_train_aligned.to_csv(base_path + 'all_train_aligned.csv')

all_test_aligned.to_csv(base_path + 'all_test_aligned.csv')

In [13]:
bool(re.fullmatch(r'^[–•«»!,.:\u060C\u061B\u061F\u2026]+$', '«'))

True

In [14]:
all_test_aligned

,0,1,2
0,ولكنها#3 تحركت#3,0,7.2
1,إما#3 لأنها#3 أحست#3 به#3 وإما#3 لأن#3 الوقفة#...,0,7.2
2,فرأته#3 فصبغ#3 الدم#3 وجهها#3 وارتدت#3,0,7.2
3,ولكنها#3 لم#3 تتجهم#5 له#3 وقالت#3 وفي#3 عينها...,0,7.2
4,ألك#3 هنا#3 كثير#3,0,7.2
...,...,...,...
3289,ت#3,14,9
3291,ولكنك#3 لم#3 توضح#3 لي#3 صفات#3 هؤلاء#3 الشمام...,14,9
3292,س#3,14,9
3294,إلى#3 سهرة#3 الغد#3 تصبحون#3 على#3 خير#3,14,9
